In [ ]:
from datetime import date
from itertools import product

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from app.model import DispatchConfig
from run_dispatch import run_dispatch

In [ ]:
DATES = pd.date_range(
    start="2024-06-15",
    end="2024-06-16",
    # end="2024-03-30",
    inclusive="both"
)
CONFIGS = [
    "preideal",
    "bess_preideal"
]

BESS_VALUES = {
    "BESS_1": {
        "MWh_nom": 2000,
        "hours_to_deplete": 2,
        "efficiency": 0.9,
        "min_soc": 0.05,
        "max_soc": 0.98,
        "initial_soc": 0.05,
        "charge_bid": 1E6,
        "discharge_bid": 0,
    },
}


In [ ]:

output = pd.DataFrame()
for (date_, dispatch_type) in product(DATES,CONFIGS):
    BESS = BESS_VALUES if "bess" in dispatch_type.lower() else None
    mpo = run_dispatch(DispatchConfig(dispatch_type=dispatch_type), DISPATCH_DATE=date_.date(), show_figs=True, BESS=BESS)
    # mpo["dispatch_type"] = dispatch_typex
    output = pd.concat([output, mpo])
output = output.stack().reset_index()
output.columns = ["datetime", "source", "MPO"]

In [ ]:
fig = px.line(output, x="datetime", y="MPO", color="source", title="MPO")
fig.update_layout(
    xaxis_title="Hora",
    yaxis_title="Precio [COP/MWh]",
    # width=800,
    # height=600,
    # xaxis=dict(
    #     dtick=3_600_000,
    # ),
)
fig.show()